# Raw signature verification

## Parameters

### Field

In [1]:
p = 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319

### Curve

In [2]:
ca = p - 3
cb = 27580193559959705877849011840389048093056905856361568521428707301988689241309860865136260764883745107765439761230575
gx = 26247035095799689268623156744566981891852923491109213387815615900925518854738050089022388053975719786650872476732087
gy = 8325710961489029985546751289520108179287853048861315594709205902480503199884419224438643760392947333078086511627871
go = 39402006196394479212279040100143613805079739270465446667946905279627659399113263569398956308152294913554433653942643

### Public key

In [3]:
x = 30827567487393551525402089664526449770646356836908584670673246523285555687049858098829191759343589369441587654976562
y = 2009068527738404759430491179267117270890161016114510417259692707866849715139082362044671225604392405754431155211363

### Digest

In [4]:
m = 29487358125424512498370551173628780853002944075174315746794686527735994862322

### Signature

In [5]:
r = 36742824419652648025863987403620927536315918738674799855291881974208903666047095304901170886918427806277713149969790
s = 36655663933123342092039028055267978704456567804916507890392767843061757921835920729532485652317488042964862515704415

## 256 bit words

In [6]:
def encode(n):
    hi = n // 2**256
    lo = n % 2**256
    return (hi, lo)

## Curve math

In [164]:
def mul512(a, b):
    hi = (a * b) // 2**256
    lo = (a * b) % 2**256
    return (hi, lo)

In [7]:
def fieldAdd(a, b):
    return (a + b) % p

In [8]:
def fieldSub(a, b):
    return (a - b) % p

In [9]:
def fieldMul(a, b):
    return (a * b) % p

In [10]:
def fieldInv(a):
    a = a % p

    # Euclids
    c, d = a, p
    uc, vc, ud, vd = 1, 0, 0, 1
    while c != 0:
        q, c, d = (d // c, d % c, c)
        uc, vc, ud, vd = ud - q * uc, vd - q * vc, uc, vc
        
    assert d == 1
    if ud > 0:
        return ud
    else:
        return ud + p

In [42]:
def fieldSqr(a):
    return (a ** 2) % p

In [158]:
def fieldMul2(a, b):
    ahi = a // 2**256
    alo = a % 2**256
    bhi = b // 2**256
    blo = b % 2**256
    
    hihi = ahi * bhi
    hilo = ahi * blo
    
    
    
    assert(r == fieldMul(a, b))
    return r

In [74]:
def orderAdd(a, b):
    return (a + b) % go

In [12]:
def orderSub(a, v):
    return (a - b) % go

In [13]:
def orderMul(a, b):
    return (a * b) % go

In [14]:
def orderInv(a):
    a = a % go

    # Euclids
    c, d = a, go
    uc, vc, ud, vd = 1, 0, 0, 1
    while c != 0:
        q, c, d = (d // c, d % c, c)
        uc, vc, ud, vd = ud - q * uc, vd - q * vc, uc, vc
        
    assert d == 1
    if ud > 0:
        return ud
    else:
        return ud + p

In [15]:
def curveAdd(a, b):
    ax, ay = a
    bx, by = b
    # l = (ay - by) / (ax - bx)
    l = fieldMul(fieldSub(ay, by), fieldInv(fieldSub(ax, bx)))
    # x = l * l - ax - bx
    x = fieldSub(fieldMul(l, l), fieldAdd(ax, bx))
    # y = l * (bx - x) - by
    y = fieldSub(fieldMul(l, fieldSub(bx, x)), by)
    return (x, y)

In [16]:
def curveDouble(a):
    ax, ay = a
    # l = (3 * ax * ax + ca) / (2 * ay)
    l = fieldMul(fieldAdd(fieldMul(3, fieldMul(ax, ax)), ca), fieldInv(fieldMul(2, ay)))
    # x = l * l - 2 * ax
    x = fieldSub(fieldMul(l, l), fieldMul(2, ax))
    # y = l * (ax - x) - ay
    y = fieldSub(fieldMul(l, fieldSub(ax, x)), ay)
    return (x, y)

In [17]:
def curveMul(r, a):
    assert(r != 0)
    # Double and add
    
    result = False
    coeff = a
    while r > 0:
        if r % 2 == 1:
            if result == False:
                result = coeff
            else:
                result = curveAdd(result, coeff)
        coeff = curveDouble(coeff)
        r = r // 2
    return result

## ECDSA

In [18]:
def verify(pub, m, r, s):
    c = orderInv(s)
    u1 = orderMul(m, c)
    u2 = orderMul(r, c)
    t1 = curveMul(u1, (gx, gy))
    t2 = curveMul(u2, pub)
    x, y = curveAdd(t1, t2)
    return x == r

In [19]:
verify((x, y), m, r, s)

True

## 256 bit math

In [20]:
B = 2**256

In [21]:
phi = p // B
plo = p % B

In [22]:
hex(phi)

'0xffffffffffffffffffffffffffffffff'

In [23]:
hex(plo)

'0xfffffffffffffffffffffffffffffffeffffffff0000000000000000ffffffff'

In [24]:
def encode(n):
    hi = n // B
    lo = n % B
    return(hex(hi), hex(lo))

In [25]:
encode(gy)

('0x3617de4a96262c6f5d9e98bf9292dc29',
 '0xf8f41dbd289a147ce9da3113b5f0b8c00a60b1ce1d7e819d7a431d7c90ea0e5f')

In [26]:
384*4*15000

23040000

In [27]:
hex(2**254)

'0x4000000000000000000000000000000000000000000000000000000000000000'

In [159]:
a = 599427618441585214480654826424804389148424569391432305892240124285955597636293123889101761615304932394858160794582
b = 11277274496372938598844737694877468910033331815336101620257290860148585098181308524035199845779780325184246673437934
r = 14597450047310184691802279767974112992212260558267197400177441867266218745241229512045609794656127977953453774034409

In [160]:
encode(a)

('0x3e501df64c8d7065d58eac499351e2a',
 '0xfcdc74fda6bd4980919ca5dcf51075e51e36e9442aba748d8d9931e0f1332bd6')

In [161]:
encode(b)

('0x49451a30e75e7a6a7f48519b72a60e4f',
 '0xf737d5a207bc2e493b8455c10652357e19a1044de6e3c1d680f328cb7015f4ee')

In [162]:
fieldMul(a, b)

14597450047310184691802279767974112992212260558267197400177441867266218745241229512045609794656127977953453774034409

In [163]:
fieldMul2(a, b)

0x17425bcf063c3596d3239ffd5d4ddd9f3c09a05d31f2804bf22b566ce3975858fd0175216045fb751c0e5f54dd432a82d5c442efa78c6367187bdf2e687bf843f8487c587cfdcfea6c1546b7aeb793cb05bb4ac37c30592b90a657183bf99610
0x17425bcf063c3596d3239ffd5d4ddd9f3c09a05d31f2804bf22b566ce39758584840b28edbab393cc3827ef680dc015e7104f88697af66132f037798adbefb86d79330c00aa3fe69758d22f7e79d4d3f418c7c793be085901e00905cefa2e3d0
0x5d096f3c18f0d65b4c8e7ff57537767cf0268174c7ca012fc8ad59b38e5d61612102ca3b6eace4f30e09fbda03700579c413e21a5ebd984cbc0dde62b6fbee1b5e4cc3002a8ff9a5d6348bdf9e7534fd0631f1e4ef82164078024173be8b8f4
1053742166889187823018035578655088286143052650167535096085392681164498326404673055905120468872623024141920457425343


AssertionError: 

In [66]:
(r // B, r % B)

(40995447352907514202055757621360755404,
 79653912081726419637956945954986414628610407990280932277295419255636544542185)

In [149]:
hex(fieldInv(4))

'0x3fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffbfffffffc00000000000000040000000'

In [144]:
384 * 2

768